<a href="https://colab.research.google.com/github/SANDEEP-CHAGANTI/FIRST-FMML-LAB/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [10]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [11]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33713693085852603
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1A)Averaging the validation accuracy across multiple splits of your dataset can indeed provide more consistent and reliable results compared to relying on a single validation split. This technique is known as cross-validation, and it helps to mitigate the impact of randomness and data variability on your model evaluation.

Here's how it works:

***K-Fold Cross-Validation:*** In K-fold cross-validation, you divide your dataset into K equally sized folds or subsets. You then train and evaluate your model K times, each time using a different fold as the validation set and the remaining folds as the training set. This ensures that your model is tested on multiple subsets of your data.

***Averaging Results:*** After each run of the cross-validation process, you record the validation accuracy (or any other metric of interest). Finally, you calculate the average and often also the standard deviation of these metrics across the K runs. The average provides you with an estimate of your model's performance, and the standard deviation gives you an idea of the consistency or variability of that performance.

***The benefits of using cross-validation include:***

***Reduced Variance:*** By evaluating your model on multiple subsets of your data, you reduce the chance of obtaining overly optimistic or pessimistic results that may happen with a single random split.

***Better Generalization:*** Cross-validation provides a more robust estimate of how well your model generalizes to unseen data because it's tested on multiple data partitions.

***Identifying Overfitting:*** You can use cross-validation to detect overfitting, as a model that performs significantly better on the training data compared to the validation data (in multiple splits) is likely overfitting.

However, it's important to note that cross-validation can be computationally expensive, especially if you have a large dataset or complex models. Additionally, when you use cross-validation, you should be cautious not to leak information from the test data into the training data, as this can lead to overly optimistic estimates of model performance.

In summary, averaging the validation accuracy across multiple splits using cross-validation is a valuable technique for obtaining more consistent and reliable model performance estimates, which is crucial in assessing your model's true capabilities.






2A)Cross-validation, by itself, does not directly provide a more accurate estimate of the test accuracy on unseen data. Instead, it provides a more accurate estimate of how well your model is likely to perform on new, unseen data that is similar to your training data. In other words, it helps estimate the model's generalization performance.

Here's how it works:

***Training and Validation Performance:*** Cross-validation gives you a better estimate of how your model performs on validation data, which is a subset of your training data. By averaging the validation results across multiple folds, you get a more robust indication of how well your model generalizes to data it hasn't seen during training.

***Estimating Test Performance:*** While cross-validation provides a good estimate of generalization performance, it doesn't replace the actual test on a completely held-out test dataset. The test dataset should be distinct from both the training and validation data, and it serves as a final, independent evaluation of your model's performance on unseen data. The results on this test dataset are what you would consider as the most accurate estimate of your model's performance on new, unseen data.

In practice, cross-validation helps you gain confidence in your model's expected performance and helps you tune hyperparameters, assess model complexity, and detect overfitting during the development process. It provides valuable insights into how well your model is likely to generalize. However, the final and most accurate estimate of test accuracy still comes from evaluating your model on a separate test dataset that it has not been exposed to during training or cross-validation.

So, while cross-validation doesn't directly give you the test accuracy, it plays a crucial role in helping you build and evaluate models that are likely to perform well when deployed on new, unseen data.






3A)The number of iterations, often referred to as "folds" in the context of cross-validation, can have an impact on the estimate of model performance. However, there isn't a straightforward answer to whether a higher number of iterations leads to a better estimate in all cases. The relationship between the number of iterations and the estimate's accuracy depends on various factors:

***Bias-Variance Trade-off:*** When you use k-fold cross-validation, increasing the number of iterations (k) can help reduce the variance in your estimate. Smaller values of k (e.g., 5 or 10) can lead to higher variance in your estimates because they rely on a smaller number of validation subsets, which might be more sensitive to the specific random splits in your data. Larger values of k (e.g., 20 or 30) can reduce this variance.

***Computational Cost:*** Increasing the number of iterations also increases the computational cost of cross-validation. This can be significant, especially if your dataset is large or if your model is computationally expensive to train.

***Data Availability:*** The number of iterations should be chosen considering the amount of data you have available. If you have a small dataset, you might need to use a smaller k to ensure that each validation fold has a sufficient amount of data for meaningful evaluation.

***Stability of Estimates:*** In practice, there's a diminishing return in the improvement of estimate stability as you increase k. After a certain point, increasing k further may not significantly enhance the stability of your estimates.

***Trade-off with Model Complexity:*** The choice of k can also interact with your model's complexity. If you have a very simple model, a smaller k might be sufficient for a reliable estimate. However, if your model is highly complex and prone to overfitting, a larger k might be necessary to obtain an accurate estimate of its generalization performance.

It's common to use values like 5-fold or 10-fold cross-validation as a reasonable compromise between bias and variance in many machine learning scenarios. However, the optimal choice of k can vary depending on the specific problem and dataset. It's often a good practice to perform experiments with different values of k to assess how sensitive your model's performance estimates are to changes in the number of iterations and then select the value of k that best suits your needs based on this analysis.






4A)Increasing the number of iterations (folds) in cross-validation can help to some extent when dealing with a very small training dataset or validation dataset, but it may not fully address the limitations posed by the small dataset size. Here's why:

***Benefits of Increasing Iterations:***

Improved Estimate Stability: When you increase the number of iterations in cross-validation, you are essentially creating more subsets of your data for validation. This can help stabilize your performance estimates by reducing the impact of the particular random split of data.

***More Robust Hyperparameter Tuning:*** With more iterations, you have more opportunities to perform hyperparameter tuning, which can be crucial when dealing with small datasets. It allows you to assess how different hyperparameters affect model performance more reliably.

***Limitations of Small Datasets:***

***Limited Data Diversity:*** Small datasets inherently have limited diversity, which can make it challenging to train a model that generalizes well. No amount of cross-validation iterations can address this fundamental limitation.

***Risk of Overfitting:*** In machine learning, there's a risk of overfitting to small datasets, where the model may memorize the training data rather than learning meaningful patterns. Cross-validation can help detect overfitting, but it may not completely eliminate this risk.

***Reduced Power for Model Training:*** Small datasets may not provide enough information for training complex models effectively. In such cases, even with increased iterations, the model's performance may be constrained by the limitations of the dataset size.

***Choosing an Appropriate Iteration Scheme:***

When dealing with a very small dataset, it's crucial to strike a balance between the number of iterations and the size of each fold. You don't want each fold to be too small, as this can lead to unstable estimates.

You should also consider techniques like stratified sampling, which ensures that each fold maintains the same class distribution as the original dataset. This can be particularly important when working with imbalanced datasets.

***Alternative Approaches:***

If you have a very small dataset, you may want to explore techniques like data augmentation (creating synthetic data points) or transfer learning (leveraging pre-trained models) to augment your dataset and improve model performance.
In summary, while increasing the number of iterations in cross-validation can provide more stable estimates and assist in model evaluation and hyperparameter tuning, it may not completely mitigate the challenges posed by a very small training dataset or validation dataset. You should carefully consider the limitations of your dataset and explore other strategies, such as data augmentation and transfer learning, to address these limitations effectively.




